# Revenue Maximization for 3 Turbines + Solar (15‑min)

This notebook helps you plan dispatch for **three turbine power plants** and **solar** to **maximize revenue** while:

- Avoiding forced full-capacity injection when it produces **zero-revenue energy**
- Accounting for **turbine start-up costs**
- Penalizing **deviation from a target schedule** (e.g., nomination or plant target)

It works in **15-minute intervals**, reads your **solar forecast CSV**, and integrates with your own **`revenue_function`** (if provided).

> ⚙️ **Two solving modes**

1. **Black-box optimizer (default, no extra packages)** — uses simulated annealing to maximize whatever your `revenue_function` returns, minus deviation and start costs. Works even if the revenue function is non-linear or proprietary.
2. **MILP with PuLP (optional)** — if you have linear tariffs and PuLP available, you can switch to an exact MILP model.

Feel free to adapt parameter cells (capacities, ramps, costs, penalties) to your plants.

## 1) Setup & Inputs
Provide file paths, plant parameters, and penalty coefficients.

**Required:**
- Solar forecast CSV with 15-min data (columns like: `timestamp, solar_forecast_mw`).

**Optional:**
- Target schedule CSV (columns like: `timestamp, target_mw`). If missing, target=0.
- Time-varying energy price CSV (columns like: `timestamp, price_per_mwh`). Only used by fallback revenue function if your `revenue_function` is not found.
- Your own `revenue_function.py` in the same folder. It should expose a callable named `revenue_function` or `calculate_revenue` or `compute_revenue`. The function may accept either the full dispatch DataFrame or arrays; see the integration wrapper below.


In [ ]:
from __future__ import annotations
import math, random, importlib, os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Paths (edit these) ----
SOLAR_CSV = 'solar_forecast_15min.csv'  # <-- replace with your file
TARGET_CSV = None  # e.g., 'target_schedule_15min.csv' or None
PRICE_CSV  = None  # optional fallback for revenue if no revenue_function provided

# ---- Plant parameters (edit to match your units; assume MW and $) ----
# Three turbines (example). If you have a file per plant, you can parse them here instead.
TURBINES = [
    {
        'name': 'GT1',
        'p_min': 5.0,          # MW when ON
        'p_max': 25.0,         # MW
        'ramp_up': 8.0,        # MW per 15-min
        'ramp_down': 8.0,      # MW per 15-min
        'startup_cost': 500.0, # $ per start
        'initial_on': 0,       # 0 or 1
        'initial_p': 0.0
    },
    {
        'name': 'GT2',
        'p_min': 5.0,
        'p_max': 30.0,
        'ramp_up': 10.0,
        'ramp_down': 10.0,
        'startup_cost': 600.0,
        'initial_on': 0,
        'initial_p': 0.0
    },
    {
        'name': 'GT3',
        'p_min': 3.0,
        'p_max': 15.0,
        'ramp_up': 6.0,
        'ramp_down': 6.0,
        'startup_cost': 350.0,
        'initial_on': 0,
        'initial_p': 0.0
    }
]

# Deviation penalty coefficient ($ per MWh absolute deviation from target)
LAMBDA_DEV = 50.0

# Soft penalty for violating ramp or bounds during heuristic (kept high to discourage)
BIG_M_PENALTY = 1e6

# Random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Optimization settings (annealing)
MAX_ITERS = 20000          # total neighbor evaluations
INIT_T = 1.0               # initial temperature (accept some worse moves)
FINAL_T = 0.001            # final temperature
ANNEAL_STEPS = 1000        # steps for temperature decay
NEIGHBOR_SCALE = 0.25      # fraction of p_max for random adjustments

# If you want to try PuLP MILP later, set to True (requires installed PuLP and linear revenue)
TRY_PULP_MILP = False


## 2) Load Data (15‑min) & Validate
This will read the solar forecast and (optionally) target and price series, and align them to a common 15-minute index.

In [ ]:
def _read_time_series_csv(path, ts_col='timestamp'):
    df = pd.read_csv(path)
    # Flexible timestamp column detection
    if ts_col not in df.columns:
        for c in df.columns:
            if 'time' in c.lower() or 'date' in c.lower():
                ts_col = c
                break
    df[ts_col] = pd.to_datetime(df[ts_col])
    df = df.sort_values(ts_col).set_index(ts_col)
    return df

# Load solar forecast
solar_df = _read_time_series_csv(SOLAR_CSV)
# Try to detect forecast column
solar_col = None
for c in solar_df.columns:
    if 'forecast' in c.lower() and 'solar' in c.lower():
        solar_col = c
        break
if solar_col is None:
    # fallback to first numeric column
    solar_col = solar_df.select_dtypes(include=[np.number]).columns[0]
solar_df = solar_df.rename(columns={solar_col: 'solar_forecast_mw'})[['solar_forecast_mw']].copy()

# Enforce 15-min frequency
solar_df = solar_df[~solar_df.index.duplicated(keep='first')]
freq = pd.infer_freq(solar_df.index)
if freq is None or freq.upper() not in ['15T','15MIN','15L']:
    # Reindex to the tightest 15-min grid covering the data
    full_idx = pd.date_range(solar_df.index.min(), solar_df.index.max(), freq='15T')
    solar_df = solar_df.reindex(full_idx).interpolate(limit_direction='both')

# Load optional target
if TARGET_CSV is not None:
    target_df = _read_time_series_csv(TARGET_CSV)
    tgt_col = None
    for c in target_df.columns:
        if 'target' in c.lower():
            tgt_col = c
            break
    if tgt_col is None:
        tgt_col = target_df.select_dtypes(include=[np.number]).columns[0]
    target_df = target_df.rename(columns={tgt_col: 'target_mw'})[['target_mw']].copy()
    # Align to 15-min
    target_df = target_df.reindex(solar_df.index).interpolate(limit_direction='both')
else:
    target_df = pd.DataFrame(index=solar_df.index, data={'target_mw': 0.0})

# Load optional price
if PRICE_CSV is not None:
    price_df = _read_time_series_csv(PRICE_CSV)
    pr_col = None
    for c in price_df.columns:
        if 'price' in c.lower():
            pr_col = c
            break
    if pr_col is None:
        pr_col = price_df.select_dtypes(include=[np.number]).columns[0]
    price_df = price_df.rename(columns={pr_col: 'price_per_mwh'})[['price_per_mwh']].copy()
    price_df = price_df.reindex(solar_df.index).interpolate(limit_direction='both')
else:
    price_df = pd.DataFrame(index=solar_df.index, data={'price_per_mwh': 0.0})

data = solar_df.join(target_df, how='left').join(price_df, how='left')
data.head(3)


## 3) Revenue Function Integration
We try to import your **`revenue_function`** from a local `revenue_function.py`.

- If found, we will call it in the objective.
- If not found, we use a simple fallback: **Revenue = price × net injection** per interval.

> **Expected signatures** (we detect automatically):

- `revenue_function(dispatch_df: pd.DataFrame) -> pd.Series or float`
- `calculate_revenue(dispatch_df: pd.DataFrame) -> ...`
- `compute_revenue(dispatch_df: pd.DataFrame) -> ...`

The `dispatch_df` we pass will include columns: `P_GT1, P_GT2, P_GT3, P_solar, net_injection_mw` indexed by timestamp (15‑min).

In [ ]:
# Try to import user's revenue function dynamically
rev_module = None
rev_func = None
for mod_name in ['revenue_function']:
    try:
        rev_module = importlib.import_module(mod_name)
        # Try common function names
        for fn in ['revenue_function', 'calculate_revenue', 'compute_revenue']:
            if hasattr(rev_module, fn):
                rev_func = getattr(rev_module, fn)
                break
        break
    except Exception as e:
        rev_module = None
        rev_func = None

def fallback_revenue(dispatch_df: pd.DataFrame) -> pd.Series:
    # Simple price * energy for each 15-min (MWh = MW * 0.25h)
    price = data.loc[dispatch_df.index, 'price_per_mwh']
    mwh = dispatch_df['net_injection_mw'] * 0.25
    return price * mwh

def evaluate_revenue(dispatch_df: pd.DataFrame):
    """Return per-interval revenue Series and total.
    Falls back to price * energy if no user function available.
    If user function returns a scalar, spread it evenly (for logging).
    """
    if rev_func is None:
        per = fallback_revenue(dispatch_df)
        return per, float(per.sum())
    try:
        res = rev_func(dispatch_df.copy())
        if isinstance(res, pd.Series):
            return res, float(res.sum())
        elif isinstance(res, pd.DataFrame) and 'revenue' in res.columns:
            per = res['revenue']
            return per, float(per.sum())
        elif isinstance(res, (list, np.ndarray)):
            per = pd.Series(res, index=dispatch_df.index)
            return per, float(per.sum())
        else:
            # scalar total
            total = float(res)
            per = pd.Series(total/len(dispatch_df), index=dispatch_df.index)
            return per, total
    except Exception as e:
        # Fallback if integration failed
        per = fallback_revenue(dispatch_df)
        return per, float(per.sum())


## 4) Helper Structures & Constraints
We model each turbine with on/off status and power setpoint per interval. During optimization we ensure:

- `0 ≤ P_i[t] ≤ p_max * on_i[t]` and `P_i[t] ≥ p_min * on_i[t]`
- Ramp: `|P_i[t] − P_i[t−1]| ≤ ramp`
- Solar dispatch `P_solar[t] ≤ solar_forecast[t]` (curtailment allowed)
- Start-up cost: add cost when `on_i` goes 0→1
- Deviation penalty: `LAMBDA_DEV * |net − target|`

All units are assumed in **MW** and **$**; energy per 15-min is `MW × 0.25 h`.

In [ ]:
index = data.index
T = len(index)

pmax = np.array([tb['p_max'] for tb in TURBINES])
pmin = np.array([tb['p_min'] for tb in TURBINES])
ru   = np.array([tb['ramp_up'] for tb in TURBINES])
rd   = np.array([tb['ramp_down'] for tb in TURBINES])
cstart = np.array([tb['startup_cost'] for tb in TURBINES])
init_on = np.array([tb['initial_on'] for tb in TURBINES], dtype=int)
init_p  = np.array([tb['initial_p'] for tb in TURBINES], dtype=float)

solar_forecast = data['solar_forecast_mw'].to_numpy(float)
target = data['target_mw'].to_numpy(float)

# Initial solution: all turbines OFF, solar follows min(forecast, max(target,0))
U = np.zeros((T, 3), dtype=int)
P = np.zeros((T, 3), dtype=float)
S = np.minimum(solar_forecast, np.maximum(target, 0.0))
S = np.clip(S, 0.0, solar_forecast)

# Set initial conditions at t=0 respecting ramps
U[0,:] = init_on
P[0,:] = np.where(init_on==1, np.clip(init_p, pmin, pmax), 0.0)
# Ensure feasibility of initial solar
S[0] = min(S[0], solar_forecast[0])

def project_feasible(U, P, S):
    """Project variables back into feasible region (bounds & ramps).
    Returns in-place adjusted copies.
    """
    T, N = U.shape
    # Bounds by on/off
    for t in range(T):
        for i in range(N):
            if U[t,i] <= 0:
                U[t,i] = 0
                P[t,i] = 0.0
            else:
                U[t,i] = 1
                P[t,i] = float(np.clip(P[t,i], pmin[i], pmax[i]))
        S[t] = float(np.clip(S[t], 0.0, solar_forecast[t]))
    # Ramps
    for t in range(1,T):
        for i in range(N):
            up_ok = P[t,i] - P[t-1,i] <= ru[i] + 1e-9
            dn_ok = P[t-1,i] - P[t,i] <= rd[i] + 1e-9
            if not up_ok:
                P[t,i] = P[t-1,i] + ru[i]
            if not dn_ok:
                P[t,i] = P[t-1,i] - rd[i]
            # Respect on/off after ramp adjust
            if U[t,i] == 0:
                P[t,i] = 0.0
            else:
                P[t,i] = float(np.clip(P[t,i], pmin[i], pmax[i]))
    return U, P, S

def compute_objective(U, P, S, return_components=False):
    # Build dispatch dataframe for revenue_function
    df = pd.DataFrame(index=index)
    for i, tb in enumerate(TURBINES):
        df['P_' + tb['name']] = P[:,i]
    df['P_solar'] = S
    df['net_injection_mw'] = df.filter(like='P_').sum(axis=1)

    per_rev, total_rev = evaluate_revenue(df)

    # Deviation penalty (absolute)
    dev = np.abs(df['net_injection_mw'].to_numpy() - target)
    dev_penalty = float(LAMBDA_DEV * (dev * 0.25).sum())  # $ per MWh, 15-min -> 0.25 h

    # Startup costs
    starts = np.zeros(3, dtype=int)
    for i in range(3):
        prev = init_on[i]
        for t in range(T):
            if U[t,i] == 1 and prev == 0:
                starts[i] += 1
            prev = U[t,i]
    startup_cost_total = float((starts * cstart).sum())

    # Hard constraint penalties (if any rounding violated)
    hard_pen = 0.0
    for t in range(T):
        for i in range(3):
            if U[t,i] == 0 and P[t,i] > 1e-6:
                hard_pen += BIG_M_PENALTY
            if U[t,i] == 1 and (P[t,i] < pmin[i]-1e-6 or P[t,i] > pmax[i]+1e-6):
                hard_pen += BIG_M_PENALTY
            if t>0:
                if P[t,i] - P[t-1,i] > ru[i] + 1e-6 or P[t-1,i] - P[t,i] > rd[i] + 1e-6:
                    hard_pen += BIG_M_PENALTY
        if S[t] - solar_forecast[t] > 1e-6 or S[t] < -1e-6:
            hard_pen += BIG_M_PENALTY

    total_obj = float(total_rev - dev_penalty - startup_cost_total - hard_pen)

    if return_components:
        return total_obj, {
            'total_revenue': total_rev,
            'deviation_penalty': dev_penalty,
            'startup_cost_total': startup_cost_total,
            'hard_penalty': hard_pen,
            'per_revenue': per_rev,
            'net_injection_mw': df['net_injection_mw'].to_numpy()
        }
    return total_obj

# Ensure initial is feasible
U, P, S = project_feasible(U, P, S)
obj0, comp0 = compute_objective(U, P, S, return_components=True)
obj0


## 5) Black-box Optimization (Simulated Annealing)
We improve the schedule by exploring neighboring solutions and probabilistically accepting changes. This approach works with any custom `revenue_function` (nonlinear permitted).

**Neighborhood moves:**
- Toggle a turbine on/off at a random time and adjust its power to `p_min`/`0`
- Nudge a turbine's power up/down within bounds and ramps
- Adjust solar dispatch up/down within [0, forecast]

All moves are projected back to a feasible region (bounds & ramps).

In [ ]:
def neighbor(U, P, S):
    U2 = U.copy()
    P2 = P.copy()
    S2 = S.copy()
    T, N = U2.shape
    move = random.choice(['toggle','nudge_p','nudge_solar'])
    t = random.randrange(T)
    if move == 'toggle':
        i = random.randrange(N)
        if U2[t,i] == 1:
            U2[t,i] = 0
            P2[t,i] = 0.0
        else:
            U2[t,i] = 1
            P2[t,i] = max(pmin[i], min(pmax[i], P2[t,i] if P2[t,i]>0 else pmin[i]))
    elif move == 'nudge_p':
        i = random.randrange(N)
        if U2[t,i] == 1:
            delta = (pmax[i]*NEIGHBOR_SCALE) * (2*random.random()-1)
            P2[t,i] = float(np.clip(P2[t,i] + delta, pmin[i], pmax[i]))
        else:
            # sometimes turn it on at p_min
            if random.random() < 0.2:
                U2[t,i] = 1
                P2[t,i] = pmin[i]
    else:  # nudge_solar
        delta = (np.max(solar_forecast)*NEIGHBOR_SCALE + 1e-6) * (2*random.random()-1)
        S2[t] = float(np.clip(S2[t] + delta, 0.0, solar_forecast[t]))

    # small chance to apply ramp-aware smoothing around t
    for i in range(U2.shape[1]):
        for k in [t-1, t, t+1]:
            if 0 <= k < T:
                if U2[k,i] == 0:
                    P2[k,i] = 0.0
                else:
                    P2[k,i] = float(np.clip(P2[k,i], pmin[i], pmax[i]))
    return project_feasible(U2, P2, S2)

def simulated_annealing(U, P, S, max_iters=MAX_ITERS, init_T=INIT_T, final_T=FINAL_T, steps=ANNEAL_STEPS):
    best_U, best_P, best_S = U.copy(), P.copy(), S.copy()
    best_obj = compute_objective(best_U, best_P, best_S)
    cur_U, cur_P, cur_S = best_U.copy(), best_P.copy(), best_S.copy()
    cur_obj = best_obj
    objs = [cur_obj]
    for it in range(max_iters):
        # temperature schedule
        Tcur = init_T * (final_T/init_T) ** (it/max(1,steps))
        nU, nP, nS = neighbor(cur_U, cur_P, cur_S)
        nobj = compute_objective(nU, nP, nS)
        if nobj >= cur_obj:
            cur_U, cur_P, cur_S, cur_obj = nU, nP, nS, nobj
            if nobj > best_obj:
                best_U, best_P, best_S, best_obj = nU, nP, nS, nobj
        else:
            # accept with probability exp((nobj-cur_obj)/T)
            if random.random() < math.exp((nobj-cur_obj)/max(1e-9, Tcur)):
                cur_U, cur_P, cur_S, cur_obj = nU, nP, nS, nobj
        if (it+1) % max(1, max_iters//10) == 0:
            objs.append(cur_obj)
    return best_U, best_P, best_S, best_obj, objs

best_U, best_P, best_S, best_obj, trace = simulated_annealing(U, P, S)
best_obj


## 6) Results & Diagnostics
We assemble the final schedule, compute component costs, and plot.

In [ ]:
def build_dispatch_df(U, P, S):
    out = pd.DataFrame(index=index)
    for i, tb in enumerate(TURBINES):
        out['ON_' + tb['name']] = U[:,i]
        out['P_' + tb['name']] = P[:,i]
    out['P_solar'] = S
    out['net_injection_mw'] = out.filter(regex=r'^P_').sum(axis=1)
    out['target_mw'] = target
    out['deviation_mw'] = out['net_injection_mw'] - out['target_mw']
    return out

best_dispatch = build_dispatch_df(best_U, best_P, best_S)
per_rev, total_rev = evaluate_revenue(best_dispatch[[
    'P_'+tb['name'] for tb in TURBINES
]] + ['P_solar','net_injection_mw'])
obj_val, comps = compute_objective(best_U, best_P, best_S, return_components=True)
best_dispatch['revenue'] = per_rev.reindex(best_dispatch.index).values
best_dispatch['abs_dev_mw'] = best_dispatch['deviation_mw'].abs()

# Count starts per turbine
starts = {}
for i, tb in enumerate(TURBINES):
    prev = init_on[i]
    cnt = 0
    for t in range(T):
        if best_U[t,i] == 1 and prev == 0:
            cnt += 1
        prev = best_U[t,i]
    starts[tb['name']] = cnt

summary = {
    'Objective ($)': obj_val,
    'Total revenue ($)': comps['total_revenue'],
    'Deviation penalty ($)': comps['deviation_penalty'],
    'Startup cost total ($)': comps['startup_cost_total'],
    'Total absolute deviation (MWh)': float((best_dispatch['abs_dev_mw']*0.25).sum()),
} | {('Starts ' + k): v for k,v in starts.items()}
summary


In [ ]:
# Plots
plt.figure(figsize=(12,6))
plt.plot(best_dispatch.index, best_dispatch['net_injection_mw'], label='Net injection (MW)')
plt.plot(best_dispatch.index, best_dispatch['target_mw'], label='Target (MW)', linestyle='--')
plt.step(best_dispatch.index, best_dispatch['P_solar'], where='post', label='Solar dispatched (MW)', alpha=0.7)
for i,tb in enumerate(TURBINES):
    col = 'P_' + tb['name']
    lbl = tb['name'] + ' (MW)'
    plt.step(best_dispatch.index, best_dispatch[col], where='post', label=lbl)
plt.title('Dispatch vs Target (15-min)')
plt.xlabel('Time')
plt.ylabel('MW')
plt.legend(ncol=2)
plt.grid(True, alpha=0.3)
plt.show()

plt.figure(figsize=(12,3))
plt.plot(best_dispatch.index, best_dispatch['revenue'], label='Revenue per 15-min ($)')
plt.title('Per-interval Revenue')
plt.xlabel('Time')
plt.ylabel('$')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

plt.figure(figsize=(10,3))
plt.plot(trace)
plt.title('Optimization trace (objective snapshots)')
plt.xlabel('Snapshot')
plt.ylabel('Objective ($)')
plt.grid(True, alpha=0.3)
plt.show()


## 7) Export Results
Save schedules to CSV/Excel for downstream use.

In [ ]:
OUT_CSV = 'optimized_dispatch_15min.csv'
OUT_XLSX = 'optimized_dispatch_15min.xlsx'
best_dispatch.to_csv(OUT_CSV, index_label='timestamp')
with pd.ExcelWriter(OUT_XLSX, engine='openpyxl') as writer:
    best_dispatch.to_excel(writer, sheet_name='dispatch')
    pd.DataFrame([summary]).to_excel(writer, sheet_name='summary', index=False)
OUT_CSV, OUT_XLSX


## 8) (Optional) MILP with PuLP (linear tariffs)
If your revenue is linear (e.g., time-varying price per MWh) and **PuLP** is installed, you can solve exactly.

> Set `TRY_PULP_MILP = True` in the setup cell to attempt this section. If PuLP isn't available, this cell will skip.

In [ ]:
if TRY_PULP_MILP:
    try:
        import pulp
        # Build MILP model
        model = pulp.LpProblem('RevenueMax', pulp.LpMaximize)
        N = 3
        Pvar = [[pulp.LpVariable('P_{}_{}'.format(i,t), lowBound=0) for t in range(T)] for i in range(N)]
        Uvar = [[pulp.LpVariable('U_{}_{}'.format(i,t), lowBound=0, upBound=1, cat='Binary') for t in range(T)] for i in range(N)]
        Svar = [pulp.LpVariable('S_{}'.format(t), lowBound=0) for t in range(T)]
        # Starts
        Yvar = [[pulp.LpVariable('Y_{}_{}'.format(i,t), lowBound=0, upBound=1, cat='Binary') for t in range(T)] for i in range(N)]
        # Deviation positive/negative parts for L1 penalty
        Dp = [pulp.LpVariable('Dp_{}'.format(t), lowBound=0) for t in range(T)]
        Dn = [pulp.LpVariable('Dn_{}'.format(t), lowBound=0) for t in range(T)]

        # Constraints
        for t in range(T):
            # Solar bound
            model += Svar[t] <= float(solar_forecast[t])
            # Power bounds & on/off
            for i in range(N):
                model += Pvar[i][t] <= pmax[i] * Uvar[i][t]
                model += Pvar[i][t] >= pmin[i] * Uvar[i][t]
                if t == 0:
                    # initial on/off link (approx)
                    pass
                else:
                    model += Pvar[i][t] - Pvar[i][t-1] <= ru[i]
                    model += Pvar[i][t-1] - Pvar[i][t] <= rd[i]
            # Deviation balance
            net = pulp.lpSum(Pvar[i][t] for i in range(N)) + Svar[t]
            model += net - float(target[t]) == Dp[t] - Dn[t]
        # Startup definition
        for i in range(N):
            # first period startup
            model += Yvar[i][0] >= Uvar[i][0] - init_on[i]
            for t in range(1,T):
                model += Yvar[i][t] >= Uvar[i][t] - Uvar[i][t-1]
        # Objective: price*energy - dev_penalty - startup
        price = data['price_per_mwh'].to_numpy(float)
        energy_factor = 0.25
        rev_term = pulp.lpSum( energy_factor*price[t]*( pulp.lpSum(Pvar[i][t] for i in range(N)) + Svar[t] ) for t in range(T))
        dev_term = energy_factor*LAMBDA_DEV*pulp.lpSum(Dp[t] + Dn[t] for t in range(T))
        startup_term = pulp.lpSum(cstart[i]*Yvar[i][t] for i in range(N) for t in range(T))
        model += rev_term - dev_term - startup_term
        _ = model.solve(pulp.PULP_CBC_CMD(msg=False))
        P_m = np.array([[pulp.value(Pvar[i][t]) for i in range(N)] for t in range(T)])
        U_m = np.array([[int(round(pulp.value(Uvar[i][t]))) for i in range(N)] for t in range(T)])
        S_m = np.array([pulp.value(Svar[t]) for t in range(T)])
        milp_dispatch = build_dispatch_df(U_m, P_m, S_m)
        per_rev_m, tot_rev_m = evaluate_revenue(milp_dispatch[[
            'P_'+tb['name'] for tb in TURBINES
        ]] + ['P_solar','net_injection_mw'])
        obj_m, comps_m = compute_objective(U_m, P_m, S_m, return_components=True)
        display(milp_dispatch.head())
        print('MILP Objective:', obj_m)
    except Exception as e:
        print('PuLP MILP unavailable or failed:', e)


## 9) How to adapt this to your repository/files
- Place your `revenue_function.py` in the **same folder** as this notebook. Ensure it defines `revenue_function(...)` (or `calculate_revenue`, `compute_revenue`). The notebook will auto-detect.
- If you have **3 turbine files** with parameters or individual logic, parse those files in the **Setup & Inputs** cell and fill the `TURBINES` list.
- Make sure your **solar forecast CSV** has a timestamp column and a numeric forecast column; rename columns if needed.
- If you have a **target nomination** (to penalize deviation), provide its CSV path in `TARGET_CSV`.
- Tune `LAMBDA_DEV` to reflect your contract/imbalance costs. Increase it to keep closer to target; decrease to chase price-driven revenue.
- Tune `startup_cost` per turbine to reflect actual warm/cold start costs.

> 🧪 Tip: Use the plots and the `summary` to understand where the optimizer prefers curtailment vs. starts.